
***

<br>
Let us look at how we can implement text classification using PyTorch
<br>

The dataset is from the Tweet Sentiment Extraction challenge from Kaggle(https://www.kaggle.com/c/tweet-sentiment-extraction/overview) 
<br>

We would implement text classification using a simple embedding bag of words using PyTorch on tweet data to classify tweets as "positive","negative" or "neutral"


<br>

***


In [1]:

import os
import re
import shutil
import string


from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split




/tmp/ipykernel_137600/2856313650.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


****
Let us define methods to pre-process the review data
****

In [2]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string



def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()



In [3]:
def get_sentiment(sentiment):
    if sentiment == 'positive':
        return 2
    elif sentiment == 'negative':
        return 1
    else:
        return 0
    

In [4]:
train_data= pd.read_csv("data/tweet-sentiment-extraction/train.csv")
train_data.dropna(axis = 0, how ='any',inplace=True) 
train_data['Num_words_text'] = train_data['text'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] >2
train_data = train_data[mask]
print('-------Train data--------')
print(train_data['sentiment'].value_counts())
print(len(train_data))
print('-------------------------')
max_train_sentence_length  = train_data['Num_words_text'].max()


train_data['text'] = train_data['text'].apply(remove_emoji)
train_data['text'] = train_data['text'].apply(remove_url)
train_data['text'] = train_data['text'].apply(clean_text)

train_data['label'] = train_data['sentiment'].apply(get_sentiment)

test_data= pd.read_csv("data/tweet-sentiment-extraction/test.csv")
test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_text'] = test_data['text'].apply(lambda x:len(str(x).split())) 

max_test_sentence_length  = test_data['Num_words_text'].max()

mask = test_data['Num_words_text'] >2
test_data = test_data[mask]

print('-------Test data--------')
print(test_data['sentiment'].value_counts())
print(len(test_data))
print('-------------------------')

test_data['text'] = test_data['text'].apply(remove_emoji)
test_data['text'] = test_data['text'].apply(remove_url)
test_data['text'] = test_data['text'].apply(clean_text)

test_data['label'] = test_data['sentiment'].apply(get_sentiment)

print('Train Max Sentence Length :'+str(max_train_sentence_length))
print('Test Max Sentence Length :'+str(max_test_sentence_length))




-------Train data--------
sentiment
neutral     10704
positive     8375
negative     7673
Name: count, dtype: int64
26752
-------------------------
-------Test data--------
sentiment
neutral     1376
positive    1075
negative     983
Name: count, dtype: int64
3434
-------------------------
Train Max Sentence Length :33
Test Max Sentence Length :32


In [5]:
train_data.head(10)

,textID,text,selected_text,sentiment,Num_words_text,label
0,cb774db0d1,have responded were going,"I`d have responded, if I were going",neutral,7,0
1,549e992a42,sooo sad will miss you here san diego,Sooo SAD,negative,10,1
2,088c60f138,boss bullying,bullying me,negative,5,1
3,9642c003ef,what interview leave alone,leave me alone,negative,5,1
4,358bd9e861,sons why couldnt they put them the releases al...,"Sons of ****,",negative,14,1
5,28b57f3990,some shameless plugging for the best rangers f...,http://www.dothebouncy.com/smf - some shameles...,neutral,12,0
6,6e0c6d75b1,2am feedings for the baby are fun when all smi...,fun,positive,14,2
8,e050245fbd,both you,Both of you,neutral,3,0
9,fc2cbefa9d,journey wow just became cooler hehe that possible,Wow... u just became cooler.,positive,10,2
10,2339a9b08b,much love hopeful reckon the chances are minim...,"as much as i love to be hopeful, i reckon the ...",neutral,23,0


In [6]:
test_data.head(10)

,textID,text,sentiment,Num_words_text,label
0,f87dea47db,last session the day,neutral,6,0
1,96d74cb729,shanghai also really exciting precisely skyscr...,positive,15,2
2,eee518ae67,recession hit veronique branquinho she has qui...,negative,13,1
4,33987a8ee5,like,positive,5,2
5,726e501993,thats great weee visitors,positive,4,2
6,261932614e,think everyone hates here lol,negative,8,1
7,afa11da83f,soooooo wish could but school and myspace comp...,negative,13,1
8,e64208b4ef,and within short time the last clue all them,neutral,12,0
9,37bcad24ca,what did you get day alright havent done anyth...,neutral,18,0
10,24c92644a4,bike was put holdshould have known that argh t...,negative,12,1


***
Let us create our train,valid and test datasets
***

In [7]:
X_train, X_valid, Y_train, Y_valid= train_test_split(train_data['text'].tolist(),\
                                                      train_data['label'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['label'].tolist(),\
                                                      random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(Y_valid)))

print('Test data len:'+str(len(test_data['text'].tolist())))
print('Class distribution'+ str(Counter(test_data['label'].tolist())))


train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
test_dat=list(zip(test_data['label'].tolist(),test_data['text'].tolist()))


Train data len:21401
Class distributionCounter({0: 8563, 2: 6700, 1: 6138})
Valid data len:5351
Class distributionCounter({0: 2141, 2: 1675, 1: 1535})
Test data len:3434
Class distributionCounter({0: 1376, 2: 1075, 1: 983})


In [8]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

***
Let us create our vocabulary on train data
***

In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


Let us define our text and label preprocessing pipleines

In [10]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [11]:
text_pipeline('here is the an example')


[62, 0, 1, 0, 12881]

In [12]:
label_pipeline('1')

1

Let us define our batch collation function

In [26]:


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter =train_dat
dataloader = DataLoader(train_iter, batch_size=20, shuffle=False, collate_fn=collate_batch)

In [30]:
for idx, (label, text, offsets) in enumerate(dataloader):
    print(idx,"\n", label,"\n",text,"\nlen",len(text),"\n",offsets,"\n","\n")
    break


0 
 tensor([2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 0, 1, 1]) 
 tensor([  536,   536,   598,   187,     5,    73,   463,    33,    49,    10,
          100,   624,     1, 18453,   286, 12928,  1520,     6,     3,  3813,
          464,   286,   117,     1,    92,   229,  6945,   986,  1103,   846,
          316,   609,   247,   576,     2,   161,    24,    48,   121,   124,
           34,     9,     1,  1123,    18,  3108, 11080,     5,  1266,     1,
          272,    82,   346,     1,   309,     8,   198,    42,    19,   766,
         1626,    20,    24,     1,   547,    87,    44,  1048,  2440,   525,
          449,     7,    53,  1031,     6,   371,   134,   835,   142,     9,
          119,   475,   741,     8,    29,    37,    31,   313,     2,   166,
           59,    21,   605,  1223,   153,    26,     7,    38,  1808,   235,
          802,    85,   486,  7618,    41,  4269,    33,    49,    10,    14,
            1,   298,    16,    43,   575,  1611, 16906,     2, 181

Let us define our text classification model

In [14]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Let us create an object of our text classification class

In [16]:
train_dat

[(2, 'nite nite bday girl have fun concert'),
 (2, 'happy mothers day she liked the poem'),
 (0,
  'most experts recommend that you exercise minutes most days the week two jillian videos'),
 (1, 'entire body hurts shower hot tea and please'),
 (2, 'going home after awesome time with the peeps'),
 (1,
  'now aches chills have joined the party why couldnt the cold just away had get worse'),
 (0, 'wee are going the movies tonight see wolverine'),
 (0, 'ack course person but think knows that'),
 (0, 'hahaha yes'),
 (0, 'officially done with school til fall'),
 (0,
  'just got back from lunch and feeling more like nap order than work but will press'),
 (1, 'ugh crying again writing rant about tumblr'),
 (2,
  'happy mothers day all the moms out there special shout mommaluv and pateven though youre jehovahs witness'),
 (1, 'going miss khyy much'),
 (1, 'tired think imma bed'),
 (2, 'bellacullen13 posted and its good'),
 (0, 'kaylalee think your the only one that knows what that number means 

In [15]:
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
print(num_class)
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

3


Let us define our train and evaluate methods

In [17]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Let us create dataloaders for text,train and validation data iterators and then train our model

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR =10  # learning rate
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
test_iter2 =test_dat 
valid_iter2= valid_dat




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

In [19]:
train_dataloader

In [18]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.692


Test model on text

In [19]:
sentiment_label = {2:"Positive",
                   1: "Negative",
                   0: "Neutral"
                  }

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "soooooo wish i could, but im in school and myspace is completely blocked"
model = model.to("cpu")

print("This is a %s tweet" %sentiment_label[predict(ex_text_str, text_pipeline)])

This is a Neutral tweet
